In [1]:
from arg_utils import is_notebook, get_cfg
cfg = get_cfg()
# override variables to experiment in notebook
if is_notebook():
    cfg["gpu"] = 2
    cfg["debug"] = False
    
    cfg["model_name"] = "ViT-B/16"    
    cfg["mode"] = "adapters" # "adapters", "train_mlp_norm", "full"
    cfg["mixup_alpha"] = 0.5
    
    cfg["text2img_num_feats"] = 2
    
# overrides and calculated default vals
if cfg["batch_size"] is None:
    if cfg["model_name"] == "ViT-L/14":
        batch_size = 4 # max 32 for single GPU CL on VitB16, 4 for ViT-L/14 (9.2GB)
    elif cfg["model_name"] == "ViT-B/16":
        batch_size = 32
    elif cfg["model_name"] == "ViT-B/32":
        batch_size = 64
    else:
        batch_size = 32
    cfg["batch_size"] = batch_size
cfg["val_check_interval"] = int(cfg["val_check_interval"] * (32 / cfg["batch_size"]))
    

# seed everything using pytorch lightning
import pytorch_lightning
pytorch_lightning.seed_everything(cfg["seed"])

locals().update(cfg)

Global seed set to 0


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(cfg["gpu"])

import pandas as pd
from PIL import Image
import numpy as np

import torch

root_folder = "/raid/8wiehe/"

In [3]:
from contrastive_learning_utils import init_clip_model
clip_base_model, transform, clip_name = init_clip_model(cfg)


In [15]:
from contrastive_learning_utils import init_dm, init_lit_model, init_trainer, init_test_dms


data_module = init_dm(cfg["dataset_name"], root_folder, clip_base_model, transform, cfg, use_cl=True,
                          use_augs=cfg["use_augs"], batch_size=cfg["batch_size"], dataset_size=cfg["dataset_size"],
                          num_workers=8, pin_memory=False)


def get_all_labels(dl):
    dl = data_module.val_dataloader()
    labels = torch.cat([x[1] for x in dl], dim=0)
    return labels

%timeit get_all_labels(data_module.val_dataloader())

11.5 s ± 2.27 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
from contrastive_learning_utils import init_dm, init_lit_model, init_trainer, init_test_dms


data_module = init_dm(cfg["dataset_name"], root_folder, clip_base_model, transform, cfg, use_cl=True,
                          use_augs=cfg["use_augs"], batch_size=cfg["batch_size"], dataset_size=cfg["dataset_size"],
                          num_workers=16, pin_memory=True)

%timeit get_all_labels(data_module.val_dataloader())

6.29 s ± 494 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
from contrastive_learning_utils import init_dm, init_lit_model, init_trainer, init_test_dms

data_module = init_dm(cfg["dataset_name"], root_folder, clip_base_model, transform, cfg, use_cl=True,
                          use_augs=cfg["use_augs"], batch_size=cfg["batch_size"], dataset_size=cfg["dataset_size"],
                          num_workers=2, pin_memory=False)

# test labels in train dataloader 
import time
start = time.time()
dl = data_module.val_dataloader()
labels = torch.cat([x[1] for x in dl], dim=0)
print("Seconds: ", time.time() - start)
print(labels.shape)
print(labels.sum(dim=0))

Seconds:  8.64484429359436
torch.Size([2991, 14])
tensor([ 528.,  534.,  113.,  326.,   90.,   34.,  109.,  560., 1129.,  670.,
          22.,  194.,  112.,  726.], dtype=torch.float64)


In [ ]:
lit_model = init_lit_model(clip_base_model, test_data_modules, data_module.steps_per_epoch, data_module.label_names, cfg)
trainer = init_trainer(root_folder, cfg)
trainer.fit(lit_model, data_module)